In [8]:
import zipfile
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# 1. UNZIP THE DATASET
dataset_path = "dataset-resized.zip"
unzip_path = "dataset-resized"
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

In [9]:
# 1. SET UP DATA GENERATORS
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    os.path.join("dataset-resized", "dataset-resized"),
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    os.path.join("dataset-resized", "dataset-resized"),
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [11]:
# 2. APPLY TRANSFER LEARNING WITH RESNET50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 3. TRAIN THE MODEL
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator
)

Epoch 1/100


64/64 [==============================] - 83s 1s/step - loss: 1.7300 - accuracy: 0.2535 - val_loss: 1.6487 - val_accuracy: 0.3241
Epoch 2/100
64/64 [==============================] - 65s 1s/step - loss: 1.6462 - accuracy: 0.2984 - val_loss: 1.6659 - val_accuracy: 0.2843
Epoch 3/100
64/64 [==============================] - 65s 1s/step - loss: 1.6254 - accuracy: 0.3063 - val_loss: 1.6674 - val_accuracy: 0.2744
Epoch 4/100
64/64 [==============================] - 66s 1s/step - loss: 1.5902 - accuracy: 0.3192 - val_loss: 1.5890 - val_accuracy: 0.3559
Epoch 5/100
64/64 [==============================] - 66s 1s/step - loss: 1.5648 - accuracy: 0.3449 - val_loss: 1.6456 - val_accuracy: 0.3082
Epoch 6/100
64/64 [==============================] - 1157s 18s/step - loss: 1.5790 - accuracy: 0.3340 - val_loss: 1.6250 - val_accuracy: 0.3082
Epoch 7/100
64/64 [==============================] - 62s 965ms/step - loss: 1.5543 - accuracy: 0.3631 - val_loss: 1.5654 - val_accuracy: 0.3857
Epoch

In [ ]:
# 4. EVALUATE AND PREDICT ON TEST SET
test_generator = test_datagen.flow_from_directory(
    os.path.join("dataset-resized", "dataset-resized"),
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
# 5. VISUALIZE ACCURACY
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()